In [2]:
from __future__ import division
import pandas as pd
import seaborn as sns
from sklearn.preprocessing import scale
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
from sklearn.preprocessing import scale
import pylab as pl
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.preprocessing import Binarizer
from keras.preprocessing import image
import os
# %matplotlib notebook

In [3]:
n_rows = n_cols = 128
n_clusters = 10

In [4]:
import seaborn as sn
import numpy as np
import matplotlib.pyplot as plt

def print_results(a, b,c):
    y_train_to_clustered  = np.dstack([y, c])[0]
    clustered_tallies = np.zeros((n_clusters, n_clusters), dtype=int)
    for i in range(0, len(y_train_to_clustered)):
        clustered_tallies[y_train_to_clustered[i][1]][y_train_to_clustered[i][0]] += 1

    cluster_to_num_map = list(map(lambda x: np.argmax(x), clustered_tallies))
    clustered_tallies = sorted(clustered_tallies, key=lambda e: np.argmax(e)) 

    fig, ax = plt.subplots(1, figsize=(15,15))
    p = sn.heatmap(clustered_tallies, annot=True, fmt="d", annot_kws={"size": 10}, cmap='coolwarm', ax=ax, square=True, yticklabels=cluster_to_num_map)
    plt.xlabel('Actual', fontsize=18)
    plt.ylabel('Cluster', fontsize=18)
    p.tick_params(length=0)
    p.xaxis.tick_top()
    p.xaxis.set_label_position('top')
    plt.title('Cluster match count for each number', fontsize= 30)
        
    # purity - sum of correct in each class divided by the total number of images
    purity_sums = np.zeros((10, 1))

    for i in range(0, len(y_train_to_clustered[:])):
        if cluster_to_num_map[y_train_to_clustered[i][1]] == b[i]:
            purity_sums[cluster_to_num_map[y_train_to_clustered[i][0]]] += 1
        else:
            purity_sums[cluster_to_num_map[y_train_to_clustered[i][0]]] += 1
    
    print('Purity ', np.add.reduce(purity_sums)[0] / len(b))

In [5]:
dog_train_path="./input/cat-and-dog/training_set/dogs/"
dog_test_path="./input/cat-and-dog/test_set/dogs/"
cat_train_path="./input/cat-and-dog/training_set/cats/"
cat_test_path="./input/cat-and-dog/test_set/cats/"

dog_train=[]
for filename in os.listdir(dog_train_path):
    if filename.endswith(".jpg"):
        img=image.load_img(dog_train_path+filename,target_size=(128,128))
        dog_train.append(image.img_to_array(img))
dog_train=np.array(dog_train)

dog_test=[]
for filename in os.listdir(dog_test_path):
    if filename.endswith(".jpg"):
        img=image.load_img(dog_test_path+filename,target_size=(128,128))
        dog_test.append(image.img_to_array(img))
dog_test=np.array(dog_test)

cat_train=[]
for filename in os.listdir(cat_train_path):
    if filename.endswith(".jpg"):
        img=image.load_img(cat_train_path+filename,target_size=(128,128))
        cat_train.append(image.img_to_array(img))
cat_train=np.array(cat_train)

cat_test=[]
for filename in os.listdir(cat_test_path):
    if filename.endswith(".jpg"):
        img=image.load_img(cat_test_path+filename,target_size=(128,128))
        cat_test.append(image.img_to_array(img))
cat_test=np.array(cat_test)

print("dog_train",dog_train.shape)
print("dog_test",dog_test.shape)
print("cat_train",cat_train.shape)
print("cat_test",cat_test.shape)

dog_train (4005, 128, 128, 3)
dog_test (1012, 128, 128, 3)
cat_train (4000, 128, 128, 3)
cat_test (1011, 128, 128, 3)


In [14]:
from turtle import shape
from sklearn.cluster import KMeans
# from sklearn.model_selection import train_test_split

# raw_line=None
# if  not raw_line == None:
#     items=raw_line.split(':')
x_dog_train=dog_train.reshape(dog_train.shape[0],-1)
x_dog_test=dog_test.reshape(dog_test.shape[0],n_rows, n_cols,3)
x_cat_train=cat_train.reshape(cat_train.shape[0],-1)
x_cat_test=cat_test.reshape(cat_test.shape[0],n_rows, n_cols,3)
# x_dog_train=dog_train.reshape(dog_train.shape[0],49152)
# x_dog_test=dog_test.reshape(dog_test.shape[0],49152)

input_shape=(n_rows,n_cols,1)

x_dog_train=x_dog_train.astype('float32')
x_dog_test=x_dog_test.astype('float32')
x_dog_train /=255
x_dog_test /=255

x_cat_train=x_cat_train.astype('float32')
x_cat_test=x_cat_test.astype('float32')
x_cat_train /=255
x_cat_test /=255

k_base=KMeans(n_clusters=n_clusters)
k_base.fit([x_dog_train],[x_cat_train])

base_clustered=k_base.predict(x_dog_train,x_cat_train)
print_results(base_clustered,x_dog_train,x_cat_train)

ValueError: Found array with dim 3. KMeans expected <= 2.

In [ ]:
import keras
from keras import Model
from keras.datasets import mnist
from keras.layers import Input, Dense, Dropout, Flatten, Conv2D, MaxPooling2D, Reshape, BatchNormalization
from keras import activations
from keras.layers.advanced_activations import LeakyReLU
import numpy as np
import tensorflow as tf

In [ ]:
n_dims = 14
def get_encoder(x):
    x = Conv2D(16, (3, 3), activation=activations.relu, padding='same', name='conv2d16')(x)
    x = Flatten(name='flatten')(x)
    x = Dense(784, activation=activations.relu, name='dense1')(x)
    x = Dense(392, activation=activations.relu, name='dense2')(x)
    x = Dense(196, activation=activations.relu, name='dense3')(x)
    x = Dense(n_dims, activation=activations.relu, name='denseDim')(x)
    return x

def get_decoder(x):
    x = Dense(196, activation=activations.relu)(x)
    x = Dense(392, activation=activations.relu)(x)
    x = Dense(784, activation='sigmoid')(x)
    x = Reshape((28,28,1))(x)
    return x

x = Input(shape=(n_rows, n_cols, 1), name='input')

encoder = get_encoder(x)
decoder = get_decoder(encoder)
autoencoder = Model(x, decoder)

autoencoder.compile(loss=keras.losses.binary_crossentropy,
              optimizer=tf.optimizers.Adam(),
              metrics=['accuracy'])

autoencoder.summary()


x_dog_train=dog_train.reshape(dog_train.shape[0],-1)
x_dog_test=dog_test.reshape(dog_test.shape[0],n_rows, n_cols, 3)
x_cat_train=cat_train.reshape(cat_train.shape[0],-1)
x_cat_test=cat_test.reshape(cat_test.shape[0],n_rows, n_cols, 3)
input_shape = (n_rows, n_cols, 1)

x_dog_train = x_dog_train.astype('float32')
x_dog_test = x_dog_test.astype('float32')
x_dog_train /= 255
x_dog_test /= 255

x_cat_train = x_dog_train.astype('float32')
x_cat_test = x_dog_test.astype('float32')
x_cat_train /= 255
x_cat_test /= 255

In [ ]:
cbs = [keras.callbacks.EarlyStopping(patience=15, monitor='val_loss'), 
       keras.callbacks.ModelCheckpoint(filepath='auto_encoder_weights.h5',
       save_best_only=True)]

training_results = autoencoder.fit(x_dog_train, x_cat_train,
          batch_size=100,
          epochs=100,
          verbose=1,
          validation_data=(x_dog_test, x_cat_test),
          callbacks=cbs)
print('complete')